In [13]:
# 安装依赖（如果未安装）


# 1. 加载数据
import pandas as pd

# 读取行为数据
file_path = 'filtered_data.csv'  # 替换为你的路径
df = pd.read_csv(file_path)
# 只保留必要的列
df = df[['user_id', 'item_id','category_id', 'behavior_type']]
df=df.head(1000000)  # 仅使用前100万条数据进行测试

# 2. 筛选购买行为
df_buy = df[df['behavior_type'] == 'buy']

# 3. 构建用户-商品矩阵（隐式反馈，值设为1）
# 构建用户-商品的交互矩阵（值为购买次数）
user_item_counts = df_buy.groupby(['user_id', 'item_id']).size().unstack(fill_value=0)
user_item_matrix = user_item_counts


# 4. 计算用户之间的相似度（余弦相似度）
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

user_sim_matrix = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

# 5. 推荐函数：为某个用户推荐N个商品
def recommend_items(user_id, user_item_matrix, user_sim_df, top_n=10):
    if user_id not in user_item_matrix.index:
        print("用户不存在")
        return []

    # 获取该用户对物品的互动（购买）
    user_vector = user_item_matrix.loc[user_id]
    similar_users = user_sim_df[user_id].drop(user_id).sort_values(ascending=False)

    # 计算推荐得分：加权平均其他相似用户的评分
    scores = pd.Series(0, index=user_item_matrix.columns)
    for sim_user_id, similarity in similar_users.items():
        scores += similarity * user_item_matrix.loc[sim_user_id]
    
    # 去除已购买过的商品
    already_bought = user_vector[user_vector > 0].index
    scores = scores.drop(labels=already_bought)

    # 返回得分最高的N个商品
    return scores.sort_values(ascending=False).head(top_n)

# 6. 示例推荐
example_user = user_item_matrix.index[0]  # 任选一个用户ID
recommendations = recommend_items(example_user, user_item_matrix, user_sim_df, top_n=10)

print(f"为用户 {example_user} 推荐的商品：")
print(recommendations)


为用户 100 推荐的商品：
item_id
81         0.0
3445563    0.0
3446521    0.0
3446518    0.0
3446108    0.0
3445810    0.0
3445773    0.0
3445670    0.0
3445545    0.0
3451377    0.0
dtype: float64
